Este es el codigo de preprocesamiento de los datos para el uso de la aplicacion.

Utilizamos las bibliotecas GeoPandas, Pandas, NumPy, Matplotlib y Shapely para trabajar con datos geoespaciales.

1. **Función `multipolygon_to_polygons`**: Convierte objetos geométricos `MultiPolygon` en una lista de `Polygon`. Si el objeto no es `MultiPolygon`, genera un error. Esta función es útil para manejar datos geoespaciales complejos al simplificarlos en formas más básicas.

2. **Lectura de Datos Geoespaciales**:
   - El código lee un archivo geoespacial llamado '2017_vegetacio.gpkg' usando GeoPandas, creando un DataFrame `df_vegetacion`.
   - Luego, aplica la función `multipolygon_to_polygons` a cada elemento en la columna 'geometry' del DataFrame. Esto convierte cualquier `MultiPolygon` en `Polygon`, simplificando así la estructura de los datos.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon

def multipolygon_to_polygons(geometry):
    """
    Convierte un MultiPolygon en una lista de Polygons.

    :param geometry: Un objeto geométrico (Polygon o MultiPolygon).
    :return: Una lista de objetos Polygon.
    """
    if isinstance(geometry, MultiPolygon):
        return [poly for poly in geometry.geoms][0]
    else:
        raise TypeError("La entrada debe ser un Polygon o MultiPolygon")
    
df_vegetacion = gpd.read_file('2017_vegetacio.gpkg')
df_vegetacion['polygons'] = df_vegetacion['geometry'].apply(multipolygon_to_polygons)

Para cada zona indicada con un Poligono calculamos el centroide.

In [2]:
# Create the centroid column
df_vegetacion['centroide'] = df_vegetacion['polygons'].centroid

Ahora asignamos el barrio, districto por cada zona que encontramos en el dataset. Para ello usamos el dataset BarcelonaCiutat_SeccionsCensals.csv que contiene la informacion necesaria.

In [3]:
df_barris = gpd.read_file('BarcelonaCiutat_SeccionsCensals.csv')
gdf_barris = gpd.GeoDataFrame(df_barris, geometry=gpd.GeoSeries.from_wkt(df_barris['geometria_etrs89']))
# Create the centroid column
gdf_barris['centroide'] = gdf_barris['geometry'].centroid

Para saber el barrio al que corresponde cada zona usamos el centroide calculado previamente. Se realiza con los siguientes pasos:

1. **Función `encontrar_datos_cercanos`**:
   - **Objetivo**: Encuentra el punto más cercano en `gdf_barris` para cada punto en `df_vegetacion`.
   - **Entrada**: Una fila (`row`) de `df_vegetacion`.
   - **Proceso**:
     - Calcula las distancias entre el centroid de `row` y todos los centroides en `gdf_barris`.
     - Identifica el punto más cercano en `gdf_barris` y extrae sus datos relevantes (distrito y barrio códigos y nombres, y geometría en formato WGS84).
   - **Retorno**: Los datos del punto más cercano.

2. **Aplicación de la Función**:
   - La función `encontrar_datos_cercanos` se aplica a cada fila de `df_vegetacion` usando `.apply()`.
   - Esto genera una serie de datos con la información del punto más cercano para cada fila.

3. **Creación de Nuevas Columnas en `df_vegetacion`**:
   - Se agregan nuevas columnas a `df_vegetacion` (`codi_districte`, `nom_districte`, `codi_barri`, `nom_barri`, `geometria_wgs84`), rellenadas con la información correspondiente de los puntos más cercanos encontrados en `gdf_barris`.

Este proceso enriquece `df_vegetacion` con información detallada de ubicación basada en la proximidad a los puntos en `gdf_barris`, útil para análisis geoespacial más detallado.

In [4]:
def encontrar_datos_cercanos(row, gdf_barris):
    """
    Encuentra los datos más cercanos (codi_districte, nom_districte, codi_barri, nom_barri, geometria_wgs84) 
    para un centroid dado en dataset1.
    """
    # Calcula las distancias entre el centroid de la fila actual y todos los centroides en dataset2
    distancias = gdf_barris['centroide'].distance(row['centroide'])
    
    # Encuentra el índice del punto más cercano
    indice_minimo = np.argmin(distancias)
    
    # Retorna los datos correspondientes del punto más cercano
    punto_cercano = gdf_barris.iloc[indice_minimo]
    return (punto_cercano['codi_districte'], punto_cercano['nom_districte'], 
            punto_cercano['codi_barri'], punto_cercano['nom_barri'], 
            punto_cercano['geometria_wgs84'])

# Aplicar la función a cada fila de dataset1
datos_cercanos = df_vegetacion.apply(lambda row: encontrar_datos_cercanos(row, gdf_barris), axis=1)

# Crear nuevas columnas en dataset1
df_vegetacion['codi_districte'] = datos_cercanos.apply(lambda x: x[0])
df_vegetacion['nom_districte'] = datos_cercanos.apply(lambda x: x[1])
df_vegetacion['codi_barri'] = datos_cercanos.apply(lambda x: x[2])
df_vegetacion['nom_barri'] = datos_cercanos.apply(lambda x: x[3])
df_vegetacion['geometria_wgs84'] = datos_cercanos.apply(lambda x: x[4])

In [5]:
df_vegetacion['geometry'] = gpd.GeoSeries.from_wkt(df_vegetacion['geometria_wgs84'])
df_vegetacion['centroide'] = df_vegetacion['geometry'].centroid

Eliminamos las columnas que no son relevantes.

In [6]:
df_vegetacion['id'] = df_vegetacion.index
df_vegetacion = df_vegetacion.drop(columns=['geometry', 'polygons'])

A continuacion integramos los datos que contienene el indice de actuacion al dataset.

In [7]:
# Read dataset
df_index = pd.read_csv('data_act_index.csv')
df_vegetacion['ActuacionIndex'] = df_index['ActuacionIndex']
df_vegetacion['PercNDVINo'] = df_index['PercNDVINo']
df_vegetacion['ndviOn'] = df_index['ndviOn']

Modificamos los nombres de las columnas para el backend.

In [8]:
df_vegetacion = df_vegetacion.rename(columns={
    'codi_districte': 'code_district', 
    'nom_districte': 'name_district', 
    'codi_barri': 'code_borough', 
    'nom_barri': 'name_borough', 
    'ActuacionIndex': 'action_index',
    'PercNDVINo': 'vegetation_index',
    'ndviOn': 'vulnerability_index'
})

Pasamos las coordenadas a texto para poderlo exportar.

In [9]:
def point_to_coordinates(polygon):
    """
    Convierte un punto en coordenadas (x, y).

    :param geometry: Un objeto geométrico (Polygon o MultiPolygon).
    :return: Una lista de objetos Polygon.
    """
    return (polygon.x, polygon.y)
    
df_vegetacion['coords'] = df_vegetacion['centroide'].apply(point_to_coordinates)

Realizamos el calculo de la media de los valores de los barrios para reducir la cantidad de informacion a visuailzar en el mapa. Seguimos lo siguientes pasos:

1. **Selección de Columnas**:
   - Se crea un nuevo DataFrame `dataset1` seleccionando ciertas columnas de `df_vegetacion` relacionadas con el distrito, el barrio, índices de vegetación, y coordenadas.

2. **Conversión de Tipos de Datos**:
   - Las columnas `code_borough` y `code_district` se convierten a enteros para asegurar la consistencia en los tipos de datos.

3. **Eliminación de Duplicados**:
   - Se crea `dataset1_tmp` eliminando duplicados de `dataset1` basados en `code_borough`, manteniendo solo la primera ocurrencia. Esto resulta en un DataFrame con una única entrada por cada barrio.

4. **Selección de Columnas Específicas**:
   - `dataset1_tmp2` se crea seleccionando solo algunas columnas de `dataset1_tmp`, enfocándose en el nombre y las coordenadas del barrio, junto con identificadores y detalles del distrito.

5. **Creación de Tabla Pivote**:
   - `df_boroughs` se crea a partir de `dataset1` como una tabla pivote. Calcula el promedio de 'action_index', 'vegetation_index' y 'vulnerability_index' para cada combinación de `code_borough` y `name_borough`. 
   - Se restablece el índice para convertir los índices en columnas regulares.

6. **Fusión de DataFrames**:
   - Finalmente, `df_final` se crea fusionando `df_boroughs` con `dataset1_tmp2` en la columna 'name_borough', utilizando un join tipo 'left'. Esto combina la información promediada por barrio con los detalles específicos del barrio y del distrito.

In [ ]:
dataset1 = df_vegetacion[['id', 'name_borough', 'code_borough', 'name_district', 'code_district', 'action_index', 'vegetation_index', 'vulnerability_index', 'coords']]

dataset1['code_borough']  = dataset1['code_borough'].astype(int)
dataset1['code_district'] = dataset1['code_district'].astype(int)

dataset1_tmp = dataset1.drop_duplicates(subset=['code_borough'], keep='first')

dataset1_tmp2 =dataset1_tmp[['name_borough', 'coords', 'id', 'name_district', 'code_district']]

df_boroughs = dataset1.pivot_table(index=['code_borough', 'name_borough'], values=['action_index', 'vegetation_index', 'vulnerability_index'], aggfunc='mean')
df_boroughs = df_boroughs.reset_index()

df_final = pd.merge(df_boroughs, dataset1_tmp2, on='name_borough', how='left')

Exportamos a JSON.

In [ ]:
json_result = df_final.to_json(orient='records')
with open('output.json', 'w', encoding='utf8') as f:
    f.write(json_result)